In [218]:
import pandas as pd
import numpy as np
import json
import time
import random
import pickle

from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=360)


## https passed proxies: 	https://20.99.187.69:8443, https://34.16.139.133:8585, https://35.245.181.60:8585

In [236]:

import openai
import configparser
from  scipy.special import expit, logit, logsumexp

from gpthelpers import *

config = configparser.ConfigParser()
config.read("config.ini")
api_key = config.get('Keys','openai_api_key')

openai.api_key = api_key

In [383]:
def token_probs_toarraylist(x):
    xnew = []
    for l in x:
        l = [i for i in l if i!=None]
        xnew.append(l)

    return([np.array(i) for i in xnew])

def avg_token_logprob(x):
    tot = np.sum(x) - bias
    N = x.shape[0] - count_tokens(prefix) + 1
    return(tot)

def popularity_logprobs(subj_list):
    resp = gpt_token_probs(subj_list, prefix + " ", model = "text-curie-001")
    ylist = token_probs_toarraylist(resp)
    out = [avg_token_logprob(y) for y in ylist]
    return(out)

prefix =  "One example nearly everyone is familiar with is"
prefix_probs = gpt_token_probs(prefix, model = "text-curie-001")
bias = np.sum(token_probs_toarraylist(prefix_probs)[0])

# popularity_logprobs(["Britney Spears", "Elvis Presley", "Bob Gaudio"])



In [367]:
json_data = [] # your list with json objects (dicts)

with open('counterfact/counterfact.json') as json_file:
   json_data = json.load(json_file)

In [408]:
subjects = list(set([x['requested_rewrite']['subject'] for x in json_data]))
relations = list(set([x['requested_rewrite']['relation_id'] for x in json_data]))
print(len(relations))

34


In [409]:
## gpt thing instead
res_dict = dict()

for r in relations:
    print("scoring popularity for relation:", r)
    subjs = get_subj_by_rid(r)
    scores = popularity_logprobs(subjs)
    res_dict[r] = pd.DataFrame({"subject":subjs, "logprob":scores})



scoring popularity for relation: P1303
scoring popularity for relation: P641
scoring popularity for relation: P276
scoring popularity for relation: P159
scoring popularity for relation: P27
scoring popularity for relation: P131
scoring popularity for relation: P106
scoring popularity for relation: P413
scoring popularity for relation: P740
scoring popularity for relation: P449
scoring popularity for relation: P937
scoring popularity for relation: P463
scoring popularity for relation: P495
scoring popularity for relation: P364
scoring popularity for relation: P20
scoring popularity for relation: P190
scoring popularity for relation: P36
scoring popularity for relation: P103
scoring popularity for relation: P101
scoring popularity for relation: P176
scoring popularity for relation: P17
scoring popularity for relation: P1412
scoring popularity for relation: P138
scoring popularity for relation: P37
scoring popularity for relation: P178
scoring popularity for relation: P127
scoring popular

In [410]:
# Store data (serialize)
with open('counterfact/pop-results-curie.pickle', 'wb') as handle:
    pickle.dump(res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [413]:
res_dict["P131"].sort_values(by = "logprob", ascending=False).tail(20)

,subject,logprob
21,Stratford Army Engine Plant,-45.632642
511,Warszawa Wschodnia Osobowa railway station,-46.322001
64,32nd Indiana Monument,-46.696029
147,Servus Credit Union Place,-46.767124
707,936 ABC Hobart,-47.130675
359,Rotunde (Vienna),-47.301253
347,Kingsley Field Air National Guard Base,-47.372801
615,Losiny Ostrov National Park,-47.390660
16,Sungai Mamberamo,-47.608731
364,"New Club, Edinburgh",-47.722683


In [405]:
[x for x in json_data if x['requested_rewrite']['subject']=="Michael Jordan"]

[{'case_id': 4664,
  'pararel_idx': 23761,
  'requested_rewrite': {'prompt': '{} is a professional',
   'relation_id': 'P641',
   'target_new': {'str': 'soccer', 'id': 'Q2736'},
   'target_true': {'str': 'basketball', 'id': 'Q5372'},
   'subject': 'Michael Jordan'},
  'paraphrase_prompts': ['Casquette as above. Michael Jordan professionally plays the sport',
   'He has brown hair. Michael Jordan, the'],
  'neighborhood_prompts': ['What sport does Dennis Rodman play? They play',
   'Tim Duncan professionally plays the sport',
   'Tim Duncan plays',
   'Charles Barkley professionally plays the sport',
   'What sport does Dennis Rodman play? They play',
   'Tim Duncan, the',
   'What sport does Kevin Durant play? They play',
   'Charles Barkley plays',
   'Wilt Chamberlain professionally plays the sport',
   'Kevin Durant plays'],
  'attribute_prompts': ['Walter Zenga, the',
   'David Beckham, the',
   'Lothar Matthäus professionally plays the sport',
   'Wayne Rooney is a professional',


In [188]:
# for each relation, first find most popular thing
# 20k things / 5 per query = 4k queries

# then, score all others gainst most popular, 4 at a time
# 20k things / 4 per query = 5k queries

def find_most_popular_batch(kw_list):
    pytrends.build_payload(kw_list = kw_list)
    res = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
    out = res.loc["United States"].sort_values(ascending=False).index[0]

    return(out)

def find_most_popular(x):
    i = 1
    best = x[0]
    while i < len(x):
        l = x[i:i+4]
        kw_list = [best] + l
        best = find_most_popular_batch(kw_list)
        i+= 4
        time.sleep(1.1)

    return(best)



In [199]:


def popularity_score_batch(input_list, reference):
    pytrends.build_payload(kw_list = input_list + [reference])
    res = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
    out = res.loc["United States"]


    return(out)


def popularity_scores(x, reference):
    i = 0
    out = pd.Series()
    x.remove(reference)
    while i < len(x):
        l = x[i:i+4]
        scores = popularity_score_batch(l, reference)
        out = pd.concat([out, scores[0:4]])
        # out = out.append(z2, )
        i+= 4
        time.sleep(1.1)

    return(out)

In [121]:
s_p1303 = [x['requested_rewrite']['subject'] for x in json_data if x['requested_rewrite']['relation_id']=="P1303"]

most_pop = find_most_popular(s_p1303[0:100])
pop_scores = popularity_scores(s_p1303[0:100], most_pop)


In [134]:
# pop_scores.sort_values(ascending=False).head(10)
pop_scores[pop_scores > 0].sort_values(ascending=False)

Britney Spears    96
Britney Spears    50
The Edge          45
Elvis Presley     39
George Martin     10
Sheryl Crow        8
Cat Stevens        6
Tom Waits          5
Brian May          5
Chris Daughtry     4
John Fogerty       4
Julian Lennon      2
David Gates        1
Chris Thile        1
Peter Head         1
Nita Strauss       1
Franz Liszt        1
Name: United States, dtype: int64

In [227]:
def get_subj_by_rid(relation_id):
    subjs = [x['requested_rewrite']['subject'] for x in json_data if x['requested_rewrite']['relation_id']==relation_id]
    return(list(set(subjs)))

In [219]:
## for each relation type, find the popularity scores!
res_dict = dict()

In [229]:
for r in relations[4:]:
    print("scoring popularity for relation:", r)
    subjs = get_subj_by_rid(r)
    # subjs = random.choices(get_subj_by_rid(r), k=20)
    ref = find_most_popular(subjs)
    time.sleep(1.1)
    scores = popularity_scores(subjs, ref)
    res_dict[r] = {"ref": ref, "scores": scores}

scoring popularity for relation: P27


TooManyRequestsError: The request failed: Google returned a response with code 429

In [221]:
# Store data (serialize)
with open('counterfact/pop-results.pickle', 'wb') as handle:
    pickle.dump(res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [359]:
# # Load data (deserialize)
with open('counterfact/pop-test.pickle', 'rb') as handle:
    res_dict = pickle.load(handle)

# unserialized_data["P1303"]["scores"]

In [364]:
res_dict["P159"]["scores"].sort_values(ascending=False)

Cadillac                              100
Boston Market                          12
McGill University                       1
Indian Space Research Organisation      0
Beijing Benz                            0
Susquehanna Radio Corporation           0
DQE                                     0
Holt Renfrew                            0
Roger Dubuis                            0
Miller Brewing Company                  0
Interislander                           0
Northern & Shell                        0
Royal Mail Steam Packet Company         0
Syrian Arab Army                        0
Beykent University                      0
SKY Perfect Well Think                  0
BayernLB                                0
Russian Post                            0
IndusInd Bank                           0
Pepe Jeans                              0
dtype: int64